# Explore Edraak content types

In [1]:
from html2text import html2text
import json
import requests
import requests_cache
requests_cache.install_cache()



In [2]:

from sushichef import CRAWLING_STAGE_OUTPUT
channel_web_rsrc = json.load(open(CRAWLING_STAGE_OUTPUT,'r'))
#                                                math           algebra

# algebra 
# root_component_url = channel_web_rsrc['children'][0]['children'][1]['component_url']

root_component_ids = []
for child in channel_web_rsrc['children'][0]['children']:
    print(child['title'])
    root_component_ids.append( child['root_component_id'] )

root_component_ids

التفاضل والتكامل
الرياضيات التطبيقية (الميكانيكا)
الإحصاء و الاحتمالات
الهندسة وعلم المثلثات
الجبر و الأنماط
الأعداد والعمليات الحسابية عليها


['5b9e193f78e7f904a04379e0',
 '5b9e191a78e7f904a04379dd',
 '5a60881e6b9064043689772d',
 '5a608819f3a50d049abf68ea',
 '5a6088188c9a02049a3e69e5',
 '5a608815f3a50d049abf68e9']

In [11]:
from sushichef import  get_component_from_id, get_component_from_url

# component
root_component = get_component_from_id(root_component_ids[0])

In [12]:
# root_component

In [13]:
from collections import defaultdict


def explore_tree(root_component):

    # report data
    component_types = defaultdict(int)
    videos = []
    exercises = []

    def walk_tree(subtree):
        component_type = subtree['component_type']
        component_types[component_type] += 1

        # Imported components
        if component_type == 'ImportedComponent':
            subtree = subtree['target_component']
            walk_tree(subtree)
            return

        if component_type == 'Video':
            videos.append(subtree)
            component_id = subtree['id']
            component_url = get_component_url(component_id)
            component = get_component_from_url(component_url)
            for child in component['children']:
                walk_tree(child)

        elif component_type == 'Exercise':
            exercises.append(subtree)
            component_id = subtree['id']
            component_url = get_component_url(component_id)
            exercise = get_component_from_url(component_url)
            question_set = exercise['question_set']
            for question in question_set['children']:
                walk_tree(question)


        for child in subtree['children']:
            walk_tree(child)


    walk_tree(root_component)

    return {
        'component_types': component_types,
        'videos': videos,
        'exercises': exercises
    }



In [21]:
no_video_info_vidoes = []
    
for root_component_id in root_component_ids:
    root_component = get_component_from_id(root_component_id)
    print(root_component['title'])
    report = explore_tree(root_component)
    for k,v in report['component_types'].items():
        print(k,v)

    # check videos
    for video_id in report['videos']:
        vid = get_component_from_id(video_id['id'])
        if 'video_info' in vid:
            vid_files = vid['video_info']['encoded_videos']
            found = False
            for vf in vid_files:
                if vf['profile'] == 'youtube':
                    found = True
            if not found:
                print('Could not find youtube profile for video', vid['id'])
        else: 
            # print('Could not find video_info for video', vid['id'])
            no_video_info_vidoes.append(vid)
    
    print('\n\n')

التفاضل والتكامل
Section 1
SubSection 4
ImportedComponent 64
OnlineLesson 56
Video 56
Exercise 56
MultipleChoiceQuestion 269
Test 8



الرياضيات التطبيقية (الميكانيكا)
Section 1
SubSection 14
ImportedComponent 90
OnlineLesson 75
Video 74
Exercise 75
MultipleChoiceQuestion 259
NumericResponseQuestion 60
Test 15



الإحصاء و الاحتمالات
Section 1
SubSection 13
ImportedComponent 117
OnlineLesson 102
Video 95
Exercise 97
MultipleChoiceQuestion 419
Test 14
NumericResponseQuestion 16



الهندسة وعلم المثلثات
Section 1
SubSection 25
ImportedComponent 249
OnlineLesson 223
Video 183
Exercise 185
MultipleChoiceQuestion 781
Test 26
NumericResponseQuestion 86



الجبر و الأنماط
Section 1
SubSection 28
ImportedComponent 279
OnlineLesson 250
Video 242
Exercise 245
MultipleChoiceQuestion 1005
Test 29
NumericResponseQuestion 33



الأعداد والعمليات الحسابية عليها
Section 1
SubSection 20
ImportedComponent 194
OnlineLesson 170
Video 170
Exercise 168
NumericResponseQuestion 50
MultipleChoiceQuestion 620
T

In [22]:
missing_videos = []
for bv in no_video_info_vidoes:
    missing_videos.append(  (bv['parent_id'], bv['id']) )

missing_videos

[('5ad4a3e4f342d704a74e8119', '5ad4a40a6b9064043e25d4ce'),
 ('5acc82406b9064043d87c93f', '5acc83006b9064043d87ce78'),
 ('5acc82323a891b049fe0f3d2', '5acc83076b9064043d87ce86'),
 ('5ad4a6693a891b049e2d347f', '5ad4a6d13a891b049fe44749'),
 ('5ad4a7626b9064043d8b82d2', '5ad4a772f342d704a6255113'),
 ('5ad4a7be3a891b049e2d348f', '5ad4a7e6f342d704a74e8138'),
 ('5ad4aab6f342d704a6255259', '5ad4ab583a891b049e2d3712'),
 ('5a4c84507dd197090857fd8a', '5a4c84507dd197090857fd8c'),
 ('5ad47dd83a891b049fe42400', '5ad47df93a891b049fe42404'),
 ('5ae176678f9c14049f202f7a', '5ae177496b9064043c6521f1'),
 ('5ad4860c3a891b049fe42897', '5ad4863e6b9064043d8b5b12'),
 ('5ac6254becf6d904a026cef7', '5ac627186b9064043d860d2f'),
 ('5ac6255b6b9064043e204165', '5ac6274becf6d904a026cf64'),
 ('5ac62801ecf6d904a026cf67', '5acc92ce6b9064043e222ae9'),
 ('5ad46b2d3a891b049e2d04c1', '5ad46b4e3a891b049fe41364'),
 ('5ad46f496b9064043d8b494a', '5ad46fadf342d704a625269b'),
 ('5ad46f79f342d704a74e3eba', '5ad46fa03a891b049fe41a08'

In [8]:
len(exercises)

826

In [9]:
# for exercise_pointer in exercises:
#     exercise = get_component_from_id(exercise_pointer['id'])
#     if 'question_set' not in exercise:
#         print('exercise w/o question_set', exercise['id'])
#         continue
#     question_set = exercise['question_set']
#     question_set_children = question_set['children']
#     for question in question_set_children:
#         component_type = question['component_type']
#         if component_type != 'MultipleChoiceQuestion':
#             print('found exercise of kind', component_type, 'id', exercise['id'])


In [10]:
exercise_pointer = exercises[13]
print(exercise_pointer['id'])
exercise = get_component_from_id(exercise_pointer['id'])
# exercise

5a4c843b7dd197090857f05c


In [11]:
# exercise container is  'parent_id': '5a4c843b7dd197090857f057',
parent = get_component_from_id(exercise['parent_id'])
parent_children = parent['children']
parent['children'] = None
parent


{'id': '5a4c843b7dd197090857f057',
 'program': 15,
 'parent_id': '5a4c843b7dd197090857f048',
 'title': 'يقدر نواتج العمليّات الحسابيّة على الأعداد النسبيّة ذهنيا بدرجة معقولة',
 'accept_activity_after_due_datetime': False,
 'deleted': False,
 'full_description': None,
 'listing_description': None,
 'visibility': 'staff_and_teachers',
 'children': None,
 'created': '2018-01-03T07:20:27.257000',
 'updated': '2018-01-03T07:20:27.311000',
 'keywords': [],
 'prerequisites': [],
 'scaffolds': [],
 'license': 'all_rights_reserved',
 'audience': [],
 'eligibility_criteria': [],
 'non_eligible_view': 'hidden',
 'gamification_points_on_completion': 0,
 'quick_access_components': [],
 '_cls': 'Component.OnlineLesson',
 'classification': ['Component', 'OnlineLesson'],
 'component_type': 'OnlineLesson',
 'completed': False,
 'grade': None,
 'max_grade': None,
 'component_state': {},
 'grade_deferred': False,
 'is_eligible': True}

In [12]:
# first exercise
# parent_children[0]

In [13]:
# second exercie
# parent_children[1]

In [14]:
grandparent = get_component_from_id(parent['parent_id'])
uncles = grandparent['children']
grandparent['children'] = None
grandparent

{'id': '5a4c843b7dd197090857f048',
 'program': 15,
 'parent_id': '5a4c84347dd197090857ece8',
 'title': 'مقارنة الأعداد النسبية واجراء العمليات الأربع عليها',
 'accept_activity_after_due_datetime': False,
 'deleted': False,
 'full_description': None,
 'listing_description': None,
 'visibility': 'public',
 'children': None,
 'created': '2018-01-03T07:20:27.120000',
 'updated': '2018-03-21T12:27:37.769000',
 'keywords': [],
 'prerequisites': [],
 'scaffolds': [],
 'license': 'all_rights_reserved',
 'audience': [],
 'eligibility_criteria': [],
 'non_eligible_view': 'hidden',
 'gamification_points_on_completion': 0,
 'quick_access_components': [],
 'release_datetime': None,
 '_cls': 'Component.Section',
 'classification': ['Component', 'Section'],
 'component_type': 'Section',
 'completed': False,
 'grade': None,
 'max_grade': None,
 'component_state': {},
 'grade_deferred': False,
 'is_eligible': True}

In [15]:
# print subtree
# for u in uncles:
#     print(u['title'], u['component_type'], u['id'])
#     for uch in u['children']:
#         print('       - ', uch['title'], uch['component_type'], uch['id'])

In [16]:
ggp = get_component_from_id(grandparent['parent_id'])
guncles = ggp['children']
ggp['children'] = None
ggp

{'id': '5a4c84347dd197090857ece8',
 '_cls': 'Component.ComponentSet.MainContentTrack',
 'title': 'Main Content',
 'visibility': 'staff_and_teachers',
 'listing_description': None,
 'license': 'all_rights_reserved',
 'parent_id': None,
 'deleted': False,
 'classification': ['Component', 'ComponentSet', 'MainContentTrack'],
 'component_type': 'MainContentTrack',
 'children': None,
 'completed': False,
 'grade': None,
 'max_grade': None,
 'component_state': {},
 'grade_deferred': False,
 'is_eligible': True}

In [17]:
# print entire course

for u in guncles:
    print(u['title'], u['component_type'], u['id'])
    for uch in u['children']:
        print('       - ', uch['title'], uch['component_type'], uch['id'])
        for ugch in uch['children']:
            print('            - ', ugch['title'], ugch['component_type'], ugch['id'])


الصف السادس Section 5ad465923a891b049fe40d57
       -  تعرّف الأعداد وطرق تمثيلها والعلاقات بينها SubSection 5ad465e73a891b049fe40d58
            -  يتعرّف خانات الأعداد حتى مليار والتميز بين قيمة عدد وقيمته OnlineLesson 5ad466313a891b049fe40fde
            -  يقرأ الأعداد ويكتبها حتى مليار OnlineLesson 5ad469af6b9064043d8b3c28
            -  يقارن الأعداد حتى مليار ويرتّبها OnlineLesson 5ad469f2f342d704a74e37a3
            -  يقرّب الأعداد الطبيعيّة حتى مليون OnlineLesson 5ad46a543a891b049fe41194
            -  يتعرّف مجموعة الأعداد الصحيحة ويفهم مدلولها OnlineLesson 5ad46a663a891b049e2d024d
            -  يمثّل الأعداد الصحيحة على خطّ الأعداد OnlineLesson 5ad46a95f342d704a74e37a6
            -  يميّز بين مجموعات الأعداد الطبيعيّة والصحيحة ويتوصّل إلى علاقة الاحتواء بينهما OnlineLesson 5ad46aa46b9064043e259c51
            -  يقارن بين أعداد صحيحة ويرتّبها OnlineLesson 5ad46ab8f342d704a74e3800
            -  يتعرّّف ويجد القيمة المطلقة لعدد صحيح OnlineLesson 5ad46aaf3a891b049fe412b3
  

            -  يتعرّف بعض خواص الجذر التربيعي OnlineLesson 5add9f0c6b9064043d249164
            -  يتعرّف ويوجد الجذر التكعيبي لعدد نسبي مكعب كامل و يميّز بين الجذر التربيعي والجذر التكعيبي لعدد نسبي OnlineLesson 5add9f0b8b01ea0499724e2c
            -  يبسّط التعابير الجذريّة التي تتضمّن أعدادا حقيقيّة OnlineLesson 5add9f016b9064043c63d969
            -  يتعرف الأعداد غير النسبيّة،  يميّز بين العدد النسبي والعدد غير النسبي، يوجد القيمة التقريبيّة لعدد غير نسبي،  يمثّل عددا غير نسبيّ على خطّ الأعداد OnlineLesson 5add9f098f9c1404a0d15b8e
            -  يتعرّف مجموعة الأعداد الحقيقيّة -  يميّز بين مجموعات الأعداد الطبيعيّة والصحيحة والنسبيّة  وغير النسبيّة والحقيقيّة OnlineLesson 5add9f056b9064043c63d96a
            -  يفهم علاقة الاحتواء بين مجموعات الأعداد OnlineLesson 5add9efa8b01ea0499724e2b
            -  يقارن ويرتّب الأعداد غير النسبيّة OnlineLesson 5add9eff6b9064043c63d968
            -  يستنتج خواص التراجح (المتباينات) على العمليّات OnlineLesson 5add9ef76b9064043c63d967
         